* you need to make model pipline and testing the model with it (error)

* Saving confusion matrix in MLFLOW (Good)
 
* Need to add sqlite backend and automatic regristary the model (Good)
  
* add experiment name and add vresion (version is git version)

* add automatic select the best model and predict the model with it with new unseen dataset. (automatic select the best model by f1 is done and predict with selection model is done)
  
* Train test val spliiting in dataset (we wil try in another dataset)
  
* Make result dataframe and output file ( we wil ltry in another dataset)

* MOve all parameters to main file and remove from others

* make argv to make sure what model need to be train.  (arg.prease try is done)

* print to logger (we will try in another dataset)
  
* Make new dataset for testing deploy model

* Deployment to flask and docker
* 

In [19]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [20]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [21]:
mlflow.set_experiment("my-experiment-1")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1682395858287, experiment_id='1', last_update_time=1682395858287, lifecycle_stage='active', name='my-experiment-1', tags={}>

In [22]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1682395858287, experiment_id='1', last_update_time=1682395858287, lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [23]:
import pandas as pd
import numpy as np
# Evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score,confusion_matrix,classification_report
# MLflow
import mlflow
import mlflow.sklearn
# others
import json
import yaml
import joblib
import argparse
from urllib.parse import urlparse
import logging
import warnings
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

import sys
# caution: path[0] is reserved for script path (or '' in REPL)

# from params_loader import read_params
# from models import build_and_load_models
# from check_data_exist import check_dataset_exist
import warnings
warnings.filterwarnings("ignore")
# np.random.seed(40)


In [24]:

# SKlearn model
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import  RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from catboost import Pool, CatBoostClassifier, cv
# import lightgbm as lgb
import xgboost as xgb


def build_and_load_models():
    # Models
    Models = {
        "Logistic Regression": LogisticRegression(),                    #
        "Support Vector Classifier": SVC(),                             # Ridge, SVC, LinearSVC, Passive_AC
        "Decision Tree": DecisionTreeClassifier(max_depth=6),           #
        "KNearest": KNeighborsClassifier(n_neighbors=5),                # doesn't have model.predict_proba so I left out.
        "GaussianNB" : GaussianNB(),                                    #
        "LDA" : LinearDiscriminantAnalysis(),                           # 
        "Ridge" : RidgeClassifier(),                                    #  
        "QDA" : QuadraticDiscriminantAnalysis(),                        #
        "Bagging" : BaggingClassifier(),                                #
        "MLP" : MLPClassifier(),                                        #
        "LSVC" : LinearSVC(),                                           #  
        "BernoulliNB" : BernoulliNB(),                                  #  
        "Passive_AC" : PassiveAggressiveClassifier(),                   # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #  
        "SGB"     : GradientBoostingClassifier(n_estimators=100, random_state=9),
        "Adaboost" : AdaBoostClassifier(n_estimators=100, random_state=9, algorithm='SAMME.R', learning_rate=0.8),
        "Extra_T" : ExtraTreesClassifier(n_estimators=100, max_features=3),
        "R_forest" : RandomForestClassifier(max_samples=0.9, n_estimators=100, max_features=3),
        # "XGB" : xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05)
        }
    return Models
 

### Add lgb, catboost and Stacking and ANN


### virtualize end result



In [25]:
train_data_path = '../data/processed/undersampling_train.parquet'
test_data_path  = '../data/processed/undersampling_test.parquet'
target = 'Class'

In [26]:
sys.path.append("../src/data/")
from data_function import get_feat_and_target
def features_labels_split(train_data_path, test_data_path, target):
    train_dataset = pd.read_parquet(train_data_path)
    test_dataset = pd.read_parquet(test_data_path)
    train_features,train_label = get_feat_and_target(train_dataset, target)
    test_features,test_label = get_feat_and_target(test_dataset, target)
    return train_features, train_label, test_features, test_label

In [27]:
train_dataset = pd.read_parquet(train_data_path)
test_dataset = pd.read_parquet(test_data_path)
train_features,train_label = get_feat_and_target(train_dataset, target)
test_features,test_label = get_feat_and_target(test_dataset, target)

spliiting X and y
spliiting X and y


In [28]:
 Models = build_and_load_models()

In [29]:
Models

{'Logistic Regression': LogisticRegression(),
 'Support Vector Classifier': SVC(),
 'Decision Tree': DecisionTreeClassifier(max_depth=6),
 'KNearest': KNeighborsClassifier(),
 'GaussianNB': GaussianNB(),
 'LDA': LinearDiscriminantAnalysis(),
 'Ridge': RidgeClassifier(),
 'QDA': QuadraticDiscriminantAnalysis(),
 'Bagging': BaggingClassifier(),
 'MLP': MLPClassifier(),
 'LSVC': LinearSVC(),
 'BernoulliNB': BernoulliNB(),
 'Passive_AC': PassiveAggressiveClassifier(),
 'SGB': GradientBoostingClassifier(random_state=9),
 'Adaboost': AdaBoostClassifier(learning_rate=0.8, n_estimators=100, random_state=9),
 'Extra_T': ExtraTreesClassifier(max_features=3),
 'R_forest': RandomForestClassifier(max_features=3, max_samples=0.9)}

In [30]:
def eval_metrics(classifier, test_features, test_labels, avg_method):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    base_score   = classifier.score(test_features,test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions, average=avg_method)
    recall = recall_score(test_labels, predictions, average=avg_method)
    f1score = f1_score(test_labels, predictions, average=avg_method)
    Matrix = confusion_matrix(test_labels, predictions)
    matrix_scores = { 
        "true negative"  : Matrix[0][0],
        "false positive" : Matrix[0][1],
        "false negative" : Matrix[1][0],
        "true positive " : Matrix[1][1]
    }
    target_names = ['0','1']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    print("Confusion Matrix")
    print("---------------------","\n")
    print(f"{Matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1score)
    
    return base_score,accuracy,precision,recall,f1score,matrix_scores

In [31]:
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

i = 1
for Model_Name, classifier in Models.items():   
        with mlflow.start_run():
                print(f"{i}. {Model_Name}")
                # fit the model
                classifier.fit(train_features, train_label)
                i = i+1
                # Calculate the metrics
                base_score,accuracy,precision,recall,f1score,matrix_scores = eval_metrics(classifier, 
                                                                        test_features, 
                                                                        test_label, 
                                                                        'weighted')
                
                mlflow.log_param("Models"            , Model_Name)
                mlflow.log_params(matrix_scores)
                mlflow.log_metric("base_score"      , base_score)
                mlflow.log_metric("accuary"         , accuracy)
                mlflow.log_metric("av_precision"    , precision)
                mlflow.log_metric("recall"          , recall)
                mlflow.log_metric("f1"              , f1score)
                signature = infer_signature(test_features, classifier.predict(test_features))
                if f1score > 0.945 :
                        mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
                print("________________________________________")

1. Logistic Regression
Classification report
--------------------- 

              precision    recall  f1-score   support

           0       0.88      0.98      0.93        98
           1       0.98      0.87      0.92        99

    accuracy                           0.92       197
   macro avg       0.93      0.92      0.92       197
weighted avg       0.93      0.92      0.92       197
 

Confusion Matrix
--------------------- 

[[96  2]
 [13 86]] 

Accuracy Measures
--------------------- 

Base score:  0.9238578680203046
Accuracy:  0.9238578680203046
Precision:  0.9292483584035767
Recall:  0.9238578680203046
F1 Score:  0.923641493675378
________________________________________
2. Support Vector Classifier
Classification report
--------------------- 

              precision    recall  f1-score   support

           0       0.86      0.99      0.92        98
           1       0.99      0.84      0.91        99

    accuracy                           0.91       197
   macro avg  

In [32]:
# with mlflow.start_run():
#     lr = LogisticRegression()
#     lr.fit(train_features, train_label)
#     signature = infer_signature(test_features, lr.predict(test_features))
#     base_score,accuracy,precision,recall,f1score,matrix_scores = eval_metrics(lr, 
#                                                             test_features, 
#                                                             test_label, 
#                                                             'weighted')

#     mlflow.log_param("Models"            , lr)
#     mlflow.log_params(matrix_scores)
#     mlflow.log_metric("base_score"      , base_score)
#     mlflow.log_metric("accuary"         , accuracy)
#     mlflow.log_metric("av_precision"    , precision)
#     mlflow.log_metric("recall"          , recall)
#     mlflow.log_metric("f1"              , f1score)
#     mlflow.sklearn.log_model(lr, "logistic regression")

In [33]:
df = mlflow.search_runs(filter_string="metrics.f1 > 0.945")

In [34]:
df['run_id']

0    171028b9a18646ecaa20f0bec240a548
1    136a9367ee8645c6b982e656e06b0019
2    08bb22c78ea64a1bbadc198ca09cc03f
3    ff6268ca0af04f91b7af19e8436939e8
4    582c295b97d744e79262e2e855910a8b
5    d8e6bc5b56474696a25fd1d1279cbdb6
Name: run_id, dtype: object

In [42]:
df['params.Models']

0    R_forest
1     Extra_T
2    Adaboost
3         SGB
4         MLP
5     Bagging
Name: params.Models, dtype: object

In [45]:
run_id = [df['run_id'][1], df['params.Models'][1]]
run_id[1]

'Extra_T'

In [37]:
# run_id = '1baadc44789547928598b3214477b955'

In [48]:
import mlflow
logged_model = f'runs:/{run_id[0]}/{run_id[1]}'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
predict = loaded_model.predict(test_features)
predict

array([1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1])

In [71]:
for i in range(len(df)):
    print(i)
    run_id = df['run_id'][i]
    print(f'Run_id is :{run_id}')
    model = df['params.Models'][i]
    logged_model = f'runs:/{run_id}/{model}'

    # Load model as a PyFuncModel.
    loaded_model = mlflow.pyfunc.load_model(logged_model)

    # Predict on a Pandas DataFrame.
    import pandas as pd
    predict = loaded_model.predict(test_features)
    print(f'{model} prediciton is \n {predict}')

0
Run_id is :171028b9a18646ecaa20f0bec240a548
R_forest prediciton is 
 [1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1 0 1
 0 0 1 0 0 0 1 1 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0
 1 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 1
 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 0
 1 1 0 0 1 0 0 0 1 0 0 1]
1
Run_id is :136a9367ee8645c6b982e656e06b0019
Extra_T prediciton is 
 [1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 0 1
 0 0 1 0 0 0 1 1 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0
 1 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 1 0 1 1 1 0 0 0 1
 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 0 1 0 0
 1 1 0 0 1 0 0 0 1 0 0 1]
2
Run_id is :08bb22c78ea64a1bbadc198ca09cc03f
Adaboost pr